In [8]:
import matplotlib.pyplot as plt
import json
import os
import sys
 
# Path for spark source folder
os.environ['SPARK_HOME'] = "/Users/jonneff/spark-1.4.0-bin-hadoop2.6"
 
# Append pyspark to Python Path
sys.path.append("/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python")
 
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
# Load in the testing code and check to see if your answer is correct
# If incorrect it will report back '1 test failed' for each failed test
# Make sure to rerun any cell you change before trying the test again
from test_helper import Test
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

Read in data.  For now, on local filesystem.

In [9]:
df = sqlContext.read.json("/Users/jonneff/Desktop/DE/RC_2007-10")

In [10]:
df.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [13]:
df.head(1)

[Row(archived=True, author=u'bostich', author_flair_css_class=None, author_flair_text=None, body=u'test', controversiality=0, created_utc=u'1192450635', distinguished=None, downs=0, edited=u'false', gilded=0, id=u'c0299an', link_id=u't3_5yba3', name=u't1_c0299an', parent_id=u't3_5yba3', retrieved_on=1427426409, score=1, score_hidden=False, subreddit=u'reddit.com', subreddit_id=u't5_6', ups=1)]

In [14]:
df.take(1)

[Row(archived=True, author=u'bostich', author_flair_css_class=None, author_flair_text=None, body=u'test', controversiality=0, created_utc=u'1192450635', distinguished=None, downs=0, edited=u'false', gilded=0, id=u'c0299an', link_id=u't3_5yba3', name=u't1_c0299an', parent_id=u't3_5yba3', retrieved_on=1427426409, score=1, score_hidden=False, subreddit=u'reddit.com', subreddit_id=u't5_6', ups=1)]

In [2]:
DATABASE = 'reddit'

In [3]:
engine = create_engine("mysql+pymysql://root@localhost/" + str(DATABASE))
con = engine.connect()
dataid = 1022

In [4]:
redditDB = con.execute("SELECT * FROM comments6")

In [5]:
reddit = pd.DataFrame(redditDB.fetchall())
reddit.columns = redditDB.keys()

In [6]:
reddit.head()

,commentAuthor,commentCreated,commentID,commentLink,commentScore,postID,subreddit,postAuthor,postBody,postCreated,...,NegRaw,vNegRaw,vPos,Pos,Neg,vNeg,posNegRatio,posNegDiff,commentBody2,commentLengthSW
0,Ajunadeeps,1432770000,crmzgnx,http://www.reddit.com/r/leagueoflegends/commen...,586,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,0,0.600000,Best backdoor since gay porn.,5
1,DBlackjack21,1432780000,crn46mg,http://www.reddit.com/r/leagueoflegends/commen...,441,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,0,0.200000,[Objection!](http://gfycat.com/OptimisticSkinn...,5
2,ThePowerOfAura,1432780000,crn58hk,http://www.reddit.com/r/leagueoflegends/commen...,325,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,0,0.166667,"Dude and I thought Sion was easy to play, clea...",18
3,SkyhuntL,1432790000,crn9iqq,http://www.reddit.com/r/leagueoflegends/commen...,125,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,NaN,0.000000,"Or a driving license, that is.",7
4,offlightsedge,1432790000,crnc5oz,http://www.reddit.com/r/leagueoflegends/commen...,33,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,0,0.333333,Preparing for a wide turn like that looks like...,12


In [7]:
reddit['timeSince'].describe()

count     253519.000000
mean       32795.809746
std        57652.954846
min            0.000000
25%         9216.500000
50%        24702.000000
75%        43076.000000
max      3279309.000000
Name: timeSince, dtype: float64

In [8]:
reddit['commentScore'].describe()

count    287829.000000
mean          6.800795
std          34.962045
min        -416.000000
25%           0.000000
50%           1.000000
75%           4.000000
max        2143.000000
Name: commentScore, dtype: float64

In [9]:
redditOrig = reddit

In [10]:
redditA = reddit[['commentID', 'subreddit', 'commentScore', 'timeSince', 'commentLength', 'posNegDiff']]

In [11]:
def perc(subreddit):
    return (round(np.percentile(reddit[reddit['subreddit'] == subreddit]['commentScore'], 3, axis = 0)), round(np.percentile(reddit[reddit['subreddit'] == subreddit]['commentScore'], 97, axis = 0)))

In [12]:
LoLper = perc('leagueoflegends')
GGper = perc('GirlGamers')
picsper = perc('pics')
polper = perc('politics')

In [13]:
print(LoLper)
print(GGper)
print(picsper)
print(polper)

(-7.0, 66.0)
(0.0, 26.0)
(-5.0, 60.0)
(-6.0, 24.0)


In [14]:
def score(subreddit, per):
    data = redditA[redditA['subreddit'] == subreddit]
    data['scoreC'] = np.nan
    data['scoreC'][data['commentScore'] <= per[0]] = 0
    data['scoreC'][data['commentScore'] >= per[1]] = 1
    return data

In [15]:
LoL = score('leagueoflegends', LoLper)
GG = score('GirlGamers', GGper)
pic = score('pics', picsper)
pol = score('politics', polper)


/anaconda/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:3368: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data

In [16]:
print LoL['scoreC'].describe()
print GG['scoreC'].describe()
print pic['scoreC'].describe()
print pol['scoreC'].describe()

count    6256.000000
mean        0.468191
std         0.499027
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: scoreC, dtype: float64
count    499.000000
mean       0.386774
std        0.487500
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: scoreC, dtype: float64
count    5535.000000
mean        0.487986
std         0.499901
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: scoreC, dtype: float64
count    3763.000000
mean        0.484985
std         0.499841
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: scoreC, dtype: float64


In [17]:
redditB = pd.concat([LoL, GG, pic, pol])

In [18]:
redditB.groupby('scoreC').mean()

,commentScore,timeSince,commentLength,posNegDiff
scoreC,,,,
0,-16.081737,20443.331973,30.225986,-0.016371
1,142.337997,10461.994676,27.414966,0.005304


In [19]:
y, X = dmatrices('scoreC ~ C(subreddit) + timeSince + commentLength + posNegDiff', redditB, return_type = "dataframe")
print X.columns

Index([u'Intercept', u'C(subreddit)[T.leagueoflegends]', u'C(subreddit)[T.pics]', u'C(subreddit)[T.politics]', u'timeSince', u'commentLength', u'posNegDiff'], dtype='object')


In [20]:
X = X.rename(columns = {'C(subreddit)[T.leagueoflegends]': 'subLoL', 
                        'C(subreddit)[T.pics]': 'subPics', 
                        'C(subreddit)[T.politics]': 'subPol'})

In [21]:
y = np.ravel(y)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [23]:
model = LogisticRegression()
model = model.fit(X_train, y_train)

In [24]:
f = open('model', 'w')
cPickle.dump(model, f)
f.close()

In [25]:
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

,0,1
0,Intercept,[0.410245670233]
1,subLoL,[0.0583930466606]
2,subPics,[0.244273666339]
3,subPol,[0.100347697994]
4,timeSince,[-7.30151157008e-05]
5,commentLength,[-0.000133795596042]
6,posNegDiff,[0.0322472638634]


In [26]:
predicted = model.predict(X_test)
print predicted

[ 1.  1.  1. ...,  0.  0.  1.]


In [27]:
probs = model.predict_proba(X_test)
print probs

[[ 0.26103584  0.73896416]
 [ 0.45698479  0.54301521]
 [ 0.49617659  0.50382341]
 ..., 
 [ 0.62978508  0.37021492]
 [ 0.79438198  0.20561802]
 [ 0.27561929  0.72438071]]


In [28]:
print metrics.accuracy_score(y_test, predicted)
print metrics.roc_auc_score(y_test, probs[:, 1])

0.646433632499
0.709111871693


In [29]:
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

[[1400  806]
 [ 691 1337]]
             precision    recall  f1-score   support

        0.0       0.67      0.63      0.65      2206
        1.0       0.62      0.66      0.64      2028

avg / total       0.65      0.65      0.65      4234



In [30]:
model.predict_proba(np.array([1, 0, 0, 0, 2*60*60, 30, .001]))

array([[ 0.42780781,  0.57219219]])

In [31]:
model.predict_proba(np.array([1, 0, 0, 0, 10*60*60, 10, -.1]))

array([[ 0.85967768,  0.14032232]])

In [32]:
model.predict_proba(np.array([0, 0, 1, 0, 1*60*60, 10, 2]))

array([[ 0.38821503,  0.61178497]])